# Setting up the notebook to run calculations


The classes and functions required to run the calculation in a Jupyter notebook are imported from the source code, along with numpy, pandas and matplotlib.

In [ ]:
from pyscses.defect_species import DefectSpecies
from pyscses.set_of_sites import Set_of_Sites
from pyscses.constants import boltzmann_eV
from pyscses.calculation import Calculation, calculate_activation_energies
from pyscses.set_up_calculation import calculate_grid_offsets
from pyscses.grid import Grid

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Defining the model

The approximations that are to be applied to the calculation are defined.

```boundary_conditions``` - Either ```'dirichlet'``` where the boundaries of the calculation are fixed to 0.0, or ```'periodic'``` where the boundaries of the calculation are equivalent. 

```site_charges``` - Either ```False``` where only the charge of the defect species is considered or ```True``` where the charge of all species is considered.

```systems``` - Either ```'mott-schottky'``` where certain defect species are considered immobile and are fixed to their bulk defect distribution or ```'gouy-chapman'``` where all defect species are considered mobile and are able to redistribute.

```core_models``` - Either ```'single'```, where the core is considered using a single segregation energy at the center of the grain boundary. ```'multi-site'```, where the core is divided into layers with fixed segregation energies, when the segregation energy of a site is less that $k_BT$ or greater than $-k_BT$, the segregation energy is fixed to 0.0. Or ```False```, where all segregation energies are considered. 

```site_models``` - Either ```'site_explicit```, where all sites are considered at their lattice positions or ```continuum``` where the segregation energies are interpolated onto a regular grid.

In [ ]:
boundary_conditions = 'dirichlet'
site_charges = False
systems = 'gouy-chapman'
core_models = False
site_models = 'continuum'

### Defining variables
The variables that the calculation requires are defined.

```alpha``` - is a damping parameter used to damp the updates to the potential during every iteration to help convergence and numerical stability.

```conv``` -  is the convergence limit that the difference between the calculated potential and the damped potential must be before convergence is accepted. 

```grid_x_min / grid_x_max``` - define the region either side of the grain boundary that will be included in the calculation.

```bulk_x_min / bulk_x_max ``` - define a region of bulk which can be used to calculate bulk properties

```dielectric``` - The relative permittivity of the material. In this case Gd-doped CeO2.

```index``` - The grain boundary orientation selected for the calculation.

```b / c``` - The width and height of the cell used in the atomistic simulation when defect segreagtion energies were calculated. Specific to the grain boundary orientation.

```temp``` - Temperature (K).

In [ ]:
alpha = 0.0005

conv = 1e-8
grid_x_min = -2.0e-8
grid_x_max = +2.0e-8
bulk_x_min = -2.0e-8
bulk_x_max = -1.0e-8

dielectric = 1

index = 111

b = 5e-9
c = 5e-9

temp = [773.15]

### Defining system specific constants.

The code that is imported calculates the grain boundary properties for any system. In the cell below the properties for a given system are defined. 

```valence``` - defect charge

```site_labels``` - ```site_1``` / ```site_2``` - labels defining the species that would be occupying the site in the pure material

```defect_labels``` - ```defect_1``` / ```defect_2``` - labels defining the defect species occupying the site in the defective material

```mole_fractions``` - the bulk mole fractions of the defect species

```initial_guess``` - an initial guess for the bulk defect mole fractions, used in a minimisation to correct the output mole fractions when ```gouy_chapman``` conditions are applied. 

The values must be input in the order [ mobile defect property, immobile defect property ].

#### Example
To demonstrate how the system specific constants would be implemented using real data an example system of gadolinium doped ceria is shown.

```python
valence = [ +2.0, -1.0 ]
site labels = [ 'O', 'Ce' ]
defect_labels = [ 'Vo', 'Gd' ]
mole_fractions = [ [ 0.05, 0.2 ] ]
initial_guess = [ [ 0.05, 0.2 ] ]
```

However for the purpose of this example notebook we are not using any particular system. The site labels are defined as site_1/site_2 and the defect labels are defined as defect_1/defect_2 with equal but opposite valence and equal mole fractions.

In [ ]:
valence = [ +1.0, -1.0 ]
site_labels = ['site_1', 'site_2']
defect_labels = ['defect_1', 'defect_2']
mole_fractions = [ [ 0.2, 0.2 ] ]
initial_guess = [ [ 0.2, 0.2 ] ]

### Input data

The path to the data file is defined and stored in ```data```.

In [ ]:
data = '../input_data/example_data_2_one_seg_energies.txt'

The input for the solver is a .txt file where each line in the file corresponds to a different site.

Each line in the .txt file needs to be a string containing the following information:

**Site label** - site_1 / site_2 - A label defining the species that would be occupying the site in pure ceria. 

**Site charge** - The charge of the non-defective site species.

**$x$ coordinate** - float - The x coordinate for the position of the site.

**Defect label** - defect_1 / defect_2 -  A label defining the defect species occupying the site.

**Segregation energy** The defect segregation energy for that defect occupying that site.

For example:

```
site_2   +1.0   -2e-09    defect_2    0.0
site_1   -1.0   -2e-09    defect_1    0.0
site_2   +1.0   -1e-09    defect_2    0.0
site_1   -1.0   -1e-09    defect_1    0.0
site_2   +1.0    0.0      defect_2    0.0
site_1   -1.0    0.0      defect_1   -1.0
site_1   +1.0    1e-09    defect_1    0.0
site_2   -1.0    1e-09    defect_2    0.0
site_1   +1.0    2e-09    defect_1    0.0
site_2   -1.0    2e-09    defect_2    0.0
```

In this example system, each $x$ coordinate on a regularly spaced grid (-50 nm to +50 nm with 1 nm spacings) has one positively charged defect (```defect_one```) and one negatively charged defect (```defect_2```). These defects are equal and opposite in their valence and therefore the system is charge neutral. For simplicity, all of the defects have a segregation energy of zero, except the central positively charged defect which has a segregation energy of -1.0 eV.